In [6]:
from urllib.request import urlopen
import zipfile
from io import StringIO
from io import BytesIO
import pandas as pd
from IPython.display import display, HTML

response = urlopen(
    'https://github.com/augurysys/machine_learning_assignment/raw/master/augury_ml_assignment_2018.zip')
augury_ml_assignment_zip = response.read()

zip_file_strio = BytesIO(augury_ml_assignment_zip)
zip_file = zipfile.ZipFile(zip_file_strio)

feature_train_csv_data = BytesIO(zip_file.read('features_train.csv'))
features_train = pd.read_csv(feature_train_csv_data)

label_train_csv_data = BytesIO(zip_file.read('labels_train.csv'))
labels_train = pd.read_csv(label_train_csv_data)

feature_test_csv_data = BytesIO(zip_file.read('features_test.csv'))
features_test = pd.read_csv(feature_test_csv_data)

label_test_csv_data = BytesIO(zip_file.read('labels_test_true.csv'))
labels_test = pd.read_csv(label_test_csv_data)


display(features_train)
display(labels_train)

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,...,Feature 91,Feature 92,Feature 93,Feature 94,Feature 95,Feature 96,Feature 97,Feature 98,Feature 99,Feature 100
0,4.439813,1.111852,9.654271,15.847790,1.939385,19.641524,0.149465,-3.716539,13.982656,0.110687,...,-10.927978,-31.137819,0.090050,-4.109929,-13.090522,1.532822,5.052170,1.022600,-5.249803,8.011507
1,5.646592,0.984627,12.184933,20.473517,2.688508,24.854832,0.215365,-5.477567,17.749226,0.125854,...,-13.464064,-37.124982,0.209756,-5.041841,-16.660956,0.653725,5.848873,1.357823,-6.514128,10.076266
2,5.510505,1.537500,11.553486,22.425871,3.394388,25.614931,0.139262,-6.342006,18.259826,0.136515,...,-13.186914,-38.001452,0.268521,-5.801921,-16.191788,0.238986,5.605141,2.371446,-5.962124,10.997060
3,3.931485,0.816272,9.727030,10.763094,1.092184,16.666101,0.294288,-1.622535,11.183068,0.120075,...,-10.404408,-29.777080,0.214568,-3.165708,-12.051099,3.110172,5.533730,-0.414034,-5.813418,6.262264
4,4.796079,0.543125,11.462984,10.184339,0.148491,19.089740,0.085555,-1.108005,12.422335,0.106296,...,-12.883011,-34.300785,0.185802,-3.062539,-14.076174,4.160389,6.947012,-1.718220,-8.117579,6.379495
5,6.166876,1.240017,11.341882,26.885742,3.939389,27.599557,0.106722,-8.362732,20.729145,0.130612,...,-13.191673,-36.512197,0.237143,-5.661611,-16.949211,-2.027228,4.559953,3.580036,-5.265172,12.849646
6,4.425110,0.340365,9.824617,13.953762,1.416168,18.281698,0.089920,-2.782400,12.626480,0.075580,...,-10.989050,-27.539574,0.011699,-3.808013,-13.115025,1.053181,4.787310,-0.140214,-5.474955,6.394181
7,3.859706,1.274554,9.138864,12.944758,1.741525,17.625693,0.187943,-2.591189,11.821640,0.140878,...,-10.353678,-30.523427,0.170721,-4.033732,-11.622671,2.574657,5.262287,0.505932,-5.178832,7.053160
8,5.241284,0.445573,16.600229,1.925049,-3.097803,21.980358,0.115544,3.048757,11.955313,0.130462,...,-18.740691,-50.629081,0.261606,-2.856871,-17.880355,11.308419,12.371832,-6.386575,-15.529539,4.987647
9,6.068607,1.340242,10.462313,33.026850,6.336936,29.139510,0.102862,-11.048020,21.860864,0.129526,...,-12.315061,-32.822708,0.273716,-7.683004,-16.840961,-5.769224,2.683593,5.671082,-2.728312,14.035742


,Annotator 1,Annotator 2,Annotator 3,Annotator 4,Annotator 5
0,male,male,male,female,male
1,male,male,male,male,male
2,male,male,male,female,female
3,male,male,male,female,male
4,male,male,male,male,female
5,male,male,female,male,male
6,male,male,male,female,male
7,male,male,male,female,male
8,male,male,male,female,male
9,male,male,male,female,female


In [7]:
features_train.head()

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,...,Feature 91,Feature 92,Feature 93,Feature 94,Feature 95,Feature 96,Feature 97,Feature 98,Feature 99,Feature 100
0,4.439813,1.111852,9.654271,15.847790,1.939385,19.641524,0.149465,-3.716539,13.982656,0.110687,...,-10.927978,-31.137819,0.090050,-4.109929,-13.090522,1.532822,5.052170,1.022600,-5.249803,8.011507
1,5.646592,0.984627,12.184933,20.473517,2.688508,24.854832,0.215365,-5.477567,17.749226,0.125854,...,-13.464064,-37.124982,0.209756,-5.041841,-16.660956,0.653725,5.848873,1.357823,-6.514128,10.076266
2,5.510505,1.537500,11.553486,22.425871,3.394388,25.614931,0.139262,-6.342006,18.259826,0.136515,...,-13.186914,-38.001452,0.268521,-5.801921,-16.191788,0.238986,5.605141,2.371446,-5.962124,10.997060
3,3.931485,0.816272,9.727030,10.763094,1.092184,16.666101,0.294288,-1.622535,11.183068,0.120075,...,-10.404408,-29.777080,0.214568,-3.165708,-12.051099,3.110172,5.533730,-0.414034,-5.813418,6.262264
4,4.796079,0.543125,11.462984,10.184339,0.148491,19.089740,0.085555,-1.108005,12.422335,0.106296,...,-12.883011,-34.300785,0.185802,-3.062539,-14.076174,4.160389,6.947012,-1.718220,-8.117579,6.379495


In [8]:
labels_train.head()

,Annotator 1,Annotator 2,Annotator 3,Annotator 4,Annotator 5
0,male,male,male,female,male
1,male,male,male,male,male
2,male,male,male,female,female
3,male,male,male,female,male
4,male,male,male,male,female
